DUBIN Baptiste

In [1]:
# DUBIN Baptiste

from sklearn import datasets
import numpy as np

iris = datasets.load_iris()

1) Récupération de la Base de Donnée Iris et partitionement des élément pour crée le corpus

In [2]:
# Code prof
Ciris = np.c_[iris.data.reshape(len(iris.data), -1), iris.target.reshape(len(iris.target), -1)]
np.random.seed(0)
np.random.shuffle(Ciris)
shuffledIrisData = Ciris[ :, :iris.data.size//len(iris.data)].reshape(iris.data.shape)
shuffledIrisTarget = Ciris[ :, iris.data.size//len(iris.data) :].reshape(iris.target.shape)

# Apprentissage
appIrisData = shuffledIrisData[0:100]
appIrisTarget = shuffledIrisTarget[0:100]

# Dev
devIrisData = shuffledIrisData[100:130]
devIrisTarget = shuffledIrisTarget[100:130]

# Test
testIrisData = shuffledIrisData[130:150]
testIrisTarget = shuffledIrisTarget[130:150]

# Affichage des valeurs obtenues
debug = True

2) Phase d'apprentissage (Corpus d'apprentissage)
    - Récupération des probabilité d'apparition de chaque classe
    - Récupération des Moyennes et Variances de chaque caractéristique par classe
    - Création de la fonction de densité de probabilité de la loi normale dans le but de calculer la vraisemblance a l'aide des Moyennes et Variances

In [3]:
# Dimension des Features et Target de notre BDD
nbCarac = 4
nbClasse = 3

# Probabilité d'apparition de chaque classe
probabilite = np.zeros(nbClasse)
for i in range(nbClasse):
    probabilite[i] += (np.count_nonzero(appIrisTarget == i))/appIrisTarget.size
    print(f"Proba de {i} : {probabilite[i]}") if debug else 0


# Moyenne
moyenne = np.ndarray((nbClasse,nbCarac))
for i in range (nbClasse):
    for j in range (nbCarac):
        moyenne[i][j] = appIrisData[appIrisTarget==i][:,j].mean()

# Affichage de la Moyenne pour debug
if debug:
    print("###############################################################################")
    print(f"Moyennne : \n{iris.feature_names}")
    for i in range(nbClasse):
        print(iris.target_names[i], ':\t',moyenne[i])

# Affichage de la Moyenne transposé pour debug
moyenne = np.transpose(moyenne)
if debug:
    print("--------------------------------------------------------------------------------")
    print(f"Moyennne transposé : \n\t\t\t\t\t{iris.target_names}")
    for i in range(nbCarac):
        print(iris.feature_names[i], ':\t',moyenne[i])


# Variance
variance = np.ndarray((nbClasse,nbCarac))
for i in range (nbClasse):
    for j in range (nbCarac):
        variance[i][j] = np.sqrt(appIrisData[appIrisTarget==i][:,j].var(axis=0, ddof = 1))

# Affichage de la Variance pour debug
if debug:
    print("###############################################################################")
    print(f"Variance : \n{iris.feature_names}")
    for i in range(nbClasse):
        print(iris.target_names[i], ':\t',variance[i])


# Fonction de densité de probabilité de la loi normale permettant de calculé la vraisemblance
def f(z, var, moy):
    return (1 /
                (var * (np.sqrt(2 * np.pi)))) *\
                    (np.exp((-1 / 2) * ((z - moy) / var) ** 2))

Proba de 0 : 0.31
Proba de 1 : 0.33
Proba de 2 : 0.36
###############################################################################
Moyennne : 
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
setosa :	 [4.96129032 3.42903226 1.46451613 0.2483871 ]
versicolor :	 [5.91212121 2.78484848 4.27272727 1.33939394]
virginica :	 [6.45555556 2.92777778 5.41944444 1.98888889]
--------------------------------------------------------------------------------
Moyennne transposé : 
					['setosa' 'versicolor' 'virginica']
sepal length (cm) :	 [4.96129032 5.91212121 6.45555556]
sepal width (cm) :	 [3.42903226 2.78484848 2.92777778]
petal length (cm) :	 [1.46451613 4.27272727 5.41944444]
petal width (cm) :	 [0.2483871  1.33939394 1.98888889]
###############################################################################
Variance : 
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
setosa :	 [0.36849009 0.4100354  0.19243767 0.10605334]

3) Phase de développement (Corpus de développement)
    - Calcul de la vraisemblance
    - Choix de la classe dont la probabilité a posteriori est supérieure à celles des autres classes
    - Observation du nombre d'erreur de prédictions avec le modèle

In [4]:
print("Phase de développement")

# Calcul de la vraisemblance
vraisemblance = np.ndarray((len(devIrisTarget), nbClasse)) # (30, 3)

for i in range(len(devIrisTarget)):
    for j in range(nbClasse):
        for k in range(nbClasse):
            vraisemblance[i][j] = f(devIrisData[i][k], variance[k][j], moyenne[k][j])
        vraisemblance[i][j] *= probabilite[j]

print(f"Vraisemblance : \n {vraisemblance}") if debug else 0

# Choix de la probabilité a posteriori la plus adéquate
x = np.argmax(vraisemblance, axis=1)

# Compte le nombre de différence entre prédiction et corpus de développement
cpt = len([i for i, j in zip(x, devIrisTarget) if i != j])

if debug:
    for i in range(len(devIrisTarget)):
        print(x[i], "-->", int(devIrisTarget[i])) if x[i] == devIrisTarget[i] else print("\t", x[i], "-->", int(devIrisTarget[i]))
print("[+] Erreur = ", cpt)

Phase de développement
Vraisemblance : 
 [[2.12822788e-01 6.11274341e-23 1.21346858e-20]
 [2.03280122e-01 1.87775705e-17 4.73088910e-17]
 [6.84079456e-06 2.90889390e-01 1.62501386e-03]
 [3.46211541e-15 1.25023245e-07 1.79394808e-01]
 [6.87442210e-20 1.77411350e-14 8.35927420e-03]
 [2.20847527e-01 1.70726861e-21 1.03780037e-19]
 [2.21886123e-01 4.25491567e-20 8.41966129e-19]
 [2.20847527e-01 1.70726861e-21 1.03780037e-19]
 [7.58823881e-08 3.31093972e-01 3.54922226e-02]
 [1.99162571e-07 4.05190980e-01 2.12856512e-02]
 [1.98566922e-01 1.95296244e-24 1.34598049e-21]
 [2.03280122e-01 1.87775705e-17 4.73088910e-17]
 [5.06101526e-07 4.42477994e-01 1.21097785e-02]
 [2.21886123e-01 4.25491567e-20 8.41966129e-19]
 [4.08786007e-22 3.73225619e-18 1.01835192e-03]
 [9.60532931e-04 2.02767569e-03 2.36527765e-06]
 [1.78188192e-18 2.83751678e-12 2.61312351e-02]
 [1.99162571e-07 4.05190980e-01 2.12856512e-02]
 [2.21886123e-01 4.25491567e-20 8.41966129e-19]
 [3.45716352e-09 9.11940014e-02 1.19907143e-01]

4) Phase de Test (Corpus de Test)
    - Calcul de la vraisemblance
    - Choix de la classe dont la probabilité a posteriori est supérieure à celles des autres classes
    - Observation du nombre d'erreur de prédictions avec le modèle

In [5]:
print("Phase de Test")
# Calcul de la vraisemblance
vraisemblance = np.ndarray((len(testIrisTarget), nbClasse))

for i in range(len(testIrisTarget)):
    for j in range(nbClasse):
        for k in range(nbClasse):
            vraisemblance[i][j] = f(testIrisData[i][k], variance[k][j], moyenne[k][j])
        vraisemblance[i][j] *= probabilite[j]

print(f"Vraisemblance : \n {vraisemblance}") if debug else 0

# Choix de la probabilité a posteriori la plus adéquate
x = np.argmax(vraisemblance, axis=1)

# Compte le nombre de différence entre prédiction et corpus de test
cpt = len([i for i, j in zip(x, testIrisTarget) if i != j])

if debug:
    for i in range(len(testIrisTarget)):
        print(x[i], "-->", int(testIrisTarget[i])) if x[i] == testIrisTarget[i] else print("\t",x[i], "-->", int(testIrisTarget[i]))
print("[+] Erreur = ", cpt)

Phase de Test
Vraisemblance : 
 [[1.05264496e-11 1.08924658e-03 3.17644828e-01]
 [3.57802310e-11 3.31625098e-03 2.90490151e-01]
 [1.15745987e-09 4.72445190e-02 1.61909888e-01]
 [3.75193368e-10 2.18403707e-02 2.07394485e-01]
 [2.15839503e-01 9.46244727e-19 6.47993929e-18]
 [6.91264331e-05 6.85922116e-02 1.35655553e-04]
 [2.20788973e-13 1.94851218e-05 3.02653716e-01]
 [1.17751799e-10 9.00931954e-03 2.52009290e-01]
 [2.21886123e-01 4.25491567e-20 8.41966129e-19]
 [2.79922266e-08 2.41416080e-01 5.61403309e-02]
 [2.96610867e-06 3.74908885e-01 3.34596401e-03]
 [3.45716352e-09 9.11940014e-02 1.19907143e-01]
 [1.99162571e-07 4.05190980e-01 2.12856512e-02]
 [2.12822788e-01 6.11274341e-23 1.21346858e-20]
 [2.21886123e-01 4.25491567e-20 8.41966129e-19]
 [2.21886123e-01 4.25491567e-20 8.41966129e-19]
 [2.20788973e-13 1.94851218e-05 3.02653716e-01]
 [2.96610867e-06 3.74908885e-01 3.34596401e-03]
 [1.28635137e-20 1.18246421e-15 4.36833726e-03]
 [2.20847527e-01 1.70726861e-21 1.03780037e-19]]
2 --> 2